# Introducción a la Selección de Variables con Regresión Lineal

¡Hola! En este notebook, vamos a explorar un concepto fundamental en el Machine Learning: la **selección de variables** (o *feature selection*).

## ¿Por qué es importante la Selección de Variables?

Cuando construimos un modelo, a menudo tenemos acceso a una gran cantidad de variables o características. Sin embargo, no todas son útiles. Algunas pueden ser ruidosas o irrelevantes. La selección de variables nos ayuda a:

1.  **Mejorar el rendimiento del modelo**: Eliminar variables irrelevantes puede reducir el ruido y permitir que el modelo se enfoque en las señales importantes.
2.  **Reducir el sobreajuste (Overfitting)**: Un modelo más simple, con menos variables, es menos propenso a memorizar los datos de entrenamiento y generaliza mejor a datos nuevos.
3.  **Aumentar la interpretabilidad**: Es mucho más fácil entender un modelo que depende de 5 variables que uno que depende de 50.
4.  **Reducir el costo computacional**: Entrenar un modelo con menos variables es más rápido y consume menos recursos.

## Paso 1: Importar las librerías necesarias

Comenzaremos importando las herramientas que vamos a utilizar de `numpy` para operaciones numéricas y `sklearn` para generar datos, crear el modelo y realizar la selección de variables.

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score

## Paso 2: Crear un conjunto de datos de ejemplo

Para hacer este ejercicio más didáctico, vamos a crear nuestro propio conjunto de datos usando `make_regression`. La ventaja es que podemos controlar exactamente cuántas variables son realmente útiles.

Crearemos un dataset con:
- **20 variables en total** (`n_features=20`).
- De las cuales, **solo 5 serán informativas** (`n_informative=5`).

El objetivo será ver si nuestro método de selección de variables puede identificar correctamente estas 5 variables importantes.

In [ ]:
# Generamos el dataset sintético
X, y = make_regression(n_samples=1000, n_features=20, n_informative=5, noise=20, random_state=42)

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensiones de los datos de entrenamiento (X_train): {X_train.shape}")
print(f"Dimensiones de los datos de prueba (X_test): {X_test.shape}")

## Paso 3: Modelo Base - usamos TODAS las variables

Primero, entrenaremos un modelo de Regresión Lineal utilizando las 20 variables. Esto nos servirá como punto de comparación (baseline) para ver si la selección de variables mejora o no nuestro resultado.

In [ ]:
# Inicializamos y entrenamos el modelo de Regresión Lineal
model_full = LinearRegression()
model_full.fit(X_train, y_train)

# Hacemos predicciones en el conjunto de prueba
y_pred_full = model_full.predict(X_test)

# Evaluamos el rendimiento del modelo
mse_full = mean_squared_error(y_test, y_pred_full)
r2_full = r2_score(y_test, y_pred_full)

print("--- Resultados del Modelo con Todas las Variables ---")
print(f"Error Cuadrático Medio (MSE): {mse_full:.2f}")
print(f"Coeficiente de Determinación (R²): {r2_full:.4f}")

## Paso 4: Aplicar Selección de Variables

Ahora, aplicaremos una técnica llamada **Eliminación Recursiva de Características (Recursive Feature Elimination - RFE)**.

**¿Cómo funciona RFE?**
1.  Entrena un modelo (en nuestro caso, Regresión Lineal) con todas las variables.
2.  Calcula la importancia de cada variable (para la regresión lineal, esto se basa en los coeficientes).
3.  Elimina la variable menos importante.
4.  Repite el proceso hasta que quede el número de variables que le hemos especificado.

Le pediremos a RFE que seleccione las 5 mejores variables, ya que sabemos que ese es el número de variables informativas que creamos.

In [ ]:
# Inicializamos el modelo que usará RFE para evaluar las variables
estimator = LinearRegression()

# Inicializamos RFE para seleccionar las 5 mejores variables
selector = RFE(estimator, n_features_to_select=5, step=1)

# Aplicamos RFE sobre los datos de entrenamiento
selector = selector.fit(X_train, y_train)

# Obtenemos las máscaras de las variables seleccionadas
selected_features_mask = selector.support_

print(f"Máscara de variables seleccionadas: \n{selected_features_mask}")

# Veamos los índices de las variables que RFE ha elegido
selected_indices = np.where(selected_features_mask)[0]
print(f"\nÍndices de las variables seleccionadas: \n{selected_indices}")

## Paso 5: Modelo Mejorado - Usando solo variables seleccionadas

Ahora que RFE ha identificado las variables más importantes, vamos a:
1.  Filtrar nuestros conjuntos de datos `X_train` y `X_test` para quedarnos solo con esas columnas.
2.  Entrenar un nuevo modelo de Regresión Lineal con este subconjunto de datos.
3.  Evaluar su rendimiento y compararlo con el modelo base.

In [ ]:
# Filtramos los datos para quedarnos solo con las variables seleccionadas
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

print(f"Nuevas dimensiones de X_train_selected: {X_train_selected.shape}")

# Inicializamos y entrenamos el nuevo modelo
model_selected = LinearRegression()
model_selected.fit(X_train_selected, y_train)

# Hacemos predicciones con el nuevo modelo
y_pred_selected = model_selected.predict(X_test_selected)

# Evaluamos el rendimiento
mse_selected = mean_squared_error(y_test, y_pred_selected)
r2_selected = r2_score(y_test, y_pred_selected)

print("\n--- Resultados del Modelo con Variables Seleccionadas ---")
print(f"Error Cuadrático Medio (MSE): {mse_selected:.2f}")
print(f"Coeficiente de Determinación (R²): {r2_selected:.4f}")

## Paso 6: Conclusiones y Comparación

Finalmente, comparemos los resultados de ambos modelos.

In [ ]:
print("--- COMPARACIÓN DE MODELOS ---\n")
print(f"Modelo con TODAS las variables (20):")
print(f"  - R²: {r2_full:.4f}")
print(f"  - MSE: {mse_full:.2f}\n")

print(f"Modelo con variables SELECCIONADAS (5):")
print(f"  - R²: {r2_selected:.4f}")
print(f"  - MSE: {mse_selected:.2f}")

### Análisis de los resultados

Observando la comparación, deberíamos notar que:

- El **Coeficiente de Determinación (R²)** es muy similar en ambos modelos. Esto significa que nuestro modelo más simple (con solo 5 variables) explica la varianza de los datos casi tan bien como el modelo complejo (con 20 variables).
- El **Error Cuadrático Medio (MSE)** también es muy parecido. A veces, incluso puede ser ligeramente menor en el modelo con variables seleccionadas, lo que indicaría que hemos eliminado ruido.

**En resumen, hemos logrado un modelo:**

✅ **Igual de preciso** (o casi).
✅ **Mucho más simple** (4 veces menos variables).
✅ **Más interpretable** y **eficiente**.

Este es el poder de la selección de variables. Es un paso crucial para construir modelos de Machine Learning robustos y eficientes.